# Tutorial 10: 3D Gyroscope

A spinning top with precession and nutation using Euler angles.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mechanics_dsl import PhysicsCompiler

compiler = PhysicsCompiler()

dsl_code = r"""
\system{gyroscope}
\defvar{theta}{Nutation angle}{rad}
\defvar{phi}{Precession angle}{rad}
\defvar{psi}{Spin angle}{rad}
\parameter{I1}{1.0}{kg.m^2}
\parameter{I3}{0.5}{kg.m^2}
\parameter{m}{1.0}{kg}
\parameter{g}{9.81}{m/s^2}
\parameter{L}{0.5}{m}
\lagrangian{
    \frac{1}{2}*I1*(\dot{theta}^2 + \dot{phi}^2*\sin{theta}^2) +
    \frac{1}{2}*I3*(\dot{psi} + \dot{phi}*\cos{theta})^2 -
    m*g*L*\cos{theta}
}
\initial{theta=0.5, phi=0.0, psi=0.0, theta_dot=0.0, phi_dot=1.0, psi_dot=10.0}
"""

result = compiler.compile_dsl(dsl_code)
sol = compiler.simulate(t_span=(0, 10), num_points=1000)

In [ ]:
t = sol['t']
theta, phi, psi = sol['y'][0], sol['y'][2], sol['y'][4]

# Tip position in 3D
L = 0.5
x = L * np.sin(theta) * np.cos(phi)
y = L * np.sin(theta) * np.sin(phi)
z = L * np.cos(theta)

fig = plt.figure(figsize=(14, 5))

ax1 = fig.add_subplot(131)
ax1.plot(t, np.degrees(theta), 'b-', label='θ (nutation)')
ax1.plot(t, np.degrees(phi), 'r-', label='φ (precession)')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Angle (degrees)')
ax1.set_title('Euler Angles')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2 = fig.add_subplot(132, projection='3d')
ax2.plot(x, y, z, 'b-', lw=1)
ax2.plot([0], [0], [0], 'ko', ms=8)
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_zlabel('Z')
ax2.set_title('Gyroscope Tip Trajectory')

ax3 = fig.add_subplot(133)
ax3.plot(x, y, 'g-', lw=1)
ax3.set_xlabel('X')
ax3.set_ylabel('Y')
ax3.set_title('Top-down View (Precession)')
ax3.axis('equal')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()